In [49]:
import os 
import random
import queue
import heapq
from urllib.parse import urlparse, urljoin
import datetime
import threading
import time
import urllib.request
from bs4 import BeautifulSoup
import httplib2
import re
import requests
from urllib.parse import urlparse
import urllib.robotparser
import json
import pickle

140065837598464
None https://en.wikipedia.org/wiki/Real_numbers
140065837598464

In [50]:
#assuming 5 is the lowest and 1 is the highest priority
def prioritizer(URL,f):
    '''Take URL and returns priority from 1 to F
    Right now it like a stub function. 
    It will return a random number from 1 to f for given inputs.
    f is the no. of front queues'''
    #print(URL)
    return random.randint(1,f)

140065837598464
None https://en.wikipedia.org/wiki/File:Linear_regression.svg


FRONTIER

Frontier should use the Mercator frontier design as discussed in lecture. Preferably it should be a class and should have the given functions. Prioritizer function is a stub right now, it will return a random number between 1 to f for given URL.

In [41]:
class frontier:
# add the code for frontier here
# should have functions __init__, get_URL, add_URLs, add_to_backqueue
    
    def __init__(self, seedURLs):
      self.seedURLs = seedURLs
    
      #front queues, make a list of lists
      #use list as a queue by using append() to push and pop(0) to pop
      self.frontQueues = [[] for i in range(FRONTQUEUES)]

      #back queues, make a list of lists
      self.backQueues = [[] for i in range(BACKQUEUES)]

      #dict holding mapping of BQ to Host
      self.BQHostMapping = {}

      #heap to maintain timestamps along with the BQ
      self.heap = []

    #returns first URL from highest priority FQ
    def getFQURL(self):
      for i in range(len(self.frontQueues)):
        if len(self.frontQueues[i]) > 0:
          return self.frontQueues[i].pop(0)

    #returns URL from a BQ
    def getBQURL(self):
      #check if heap has root node then return the node
      if len(self.heap) > 0:
        root = heapq.heappop(self.heap)

        #check the delaye needed
        diff = root[0] - datetime.datetime.now()
        sleepTime = diff.total_seconds()

        #if timestamp is greater than currTime, sleep
        if sleepTime > 0:
          time.sleep(sleepTime)

        #extract the URL from BQ
        URL = self.backQueues[root[1]].pop(0)

        #check if BQ become empty
        #if BQ is empty, delete mapping from table and add a URL in BQ
        if len(self.backQueues[root[1]]) == 0:
          for key, value in self.BQHostMapping.items():
            if value == root[1]:
              del self.BQHostMapping[key]
              break

          self.addToBackQueue(root[1])
        
        #update timestamp heap for the BQ
        delay = datetime.timedelta(0,15)
        newTS = root[0] + delay
        heapq.heappush(self.heap, [newTS, root[1]])

        print("URL Removed from BQ:", URL)

        REMOVEDURLS.append(URL)
        return URL
        
      else:
        return None

    
    #takes a list of URLs and adds them to FQ based on priority
    def addURLs(self, URL):
      for i in range(len(URL)):
        priority = prioritizer(URL[i], FRONTQUEUES)
        
        #decrement priority because index of list will start from 0 not 1
        self.frontQueues[priority-1].append(URL[i])
      
      return
    
    #takes index of BQ and adds URLs unitll BQ is non-empty
    def addToBackQueue(self, BQIndex):
      
      while len(self.backQueues[BQIndex]) == 0:

        #get URL from highest priority FQ
        URL = self.getFQURL()

        #get hostname from the URL
        host = urlparse(URL).hostname

        #check if a BQ already has that host then add URL in that BQ
        #else add URL in empty BQ and update mapping table
        if host in self.BQHostMapping:
          self.backQueues[self.BQHostMapping[host]].append(URL)
        else:
          self.backQueues[BQIndex].append(URL)
          #update mapping table
          self.BQHostMapping[host] = BQIndex
        
      return

FILTER URLS

Filter the URLS that are in robots.txt files of server and the have been already processed.

In [42]:
def fetch(myFrontier):
  
  URL = myFrontier.getBQURL()

  if URL == None:
    return None

  #fetch URL
  with urllib.request.urlopen(URL) as response:
    html = response.read()
    
  #save the URL and content
  CONTENT[URL]= html

  return URL

#get URLS in proper format if not already
def fixURL(href, URL):
  
  #if URL is relative, make it absolute
  if bool(urlparse(href).netloc) == False:
    href = urljoin(URL, href)
  
  #some URLs start with // so fix them
  if href.startswith("//"):
    href = "https:" + href

  return href

#takes the domain name and link and returns whether the link can be crawled or not
def URLFilter(domain_name, href):
    
    #check if we already have the robot filename for the URL
    if domain_name in ROBOTS:
      robotFile = ROBOTS.get(domain_name)
    else:
      robotFile = "https://" + domain_name + "/robots.txt"
      ROBOTS[domain_name] = robotFile
    
    #takes robotFile name and tells if a given link can be crawled
    rp = urllib.robotparser.RobotFileParser()
    rp.set_url(robotFile)
    rp.read()
    
    canFetch = rp.can_fetch("*", href)

    return canFetch


# Run Crawler

In [43]:
# Theard task
def crawler_thread_task(myFrontier):

  i = 0
  while i < 1:
    
    URL = fetch(myFrontier)
    if URL == None:
      print("No URLs in the Frontier")
      return None

    domain_name = urlparse(URL).netloc
    soup = BeautifulSoup(requests.get(URL).content, "html.parser")
    
    linksExtracted = []
    for a_tag in soup.findAll("a"):
      href = a_tag.attrs.get("href")
      if href == "" or href is None or href[0] == "#":      #remove empty and redundant tags
        continue

      #get URLS in proper format if not already
      if href[0] != "h":
        href = fixURL(href, URL)
      
      #only process the links in proper format
      if href.startswith("https://"):
        canFetch = URLFilter(domain_name, href)

        #add URL to list of links if it is not a duplicate
        if canFetch == True:
          #has the same URL already been extracted from the webpage
          if href not in linksExtracted:
            #is the new URL already crawled
            if href not in REMOVEDURLS:
              #print(threading.get_ident())
              #print(href)
              linksExtracted.append(href)
    
    #print("NEW LINKS", *linksExtracted, sep="\n")
    #add new URLS to the frontier
    #myFrontier.addURLs(linksExtracted))
      
    i = i + 1

  a_file = open("data.pkl", "wb")
  pickle.dump(CONTENT, a_file)
  a_file.close()

  print("1000 URLs Crawled. Stopping Now!")

  return None

In [47]:
# Crawler Parameters
BACKQUEUES= 3
THREADS= BACKQUEUES*3
FRONTQUEUES= 5
WAITTIME= 15 ; # wait 15 seconds before fetching URLS from 
REMOVEDURLS = []
CONTENT = {}
ROBOTS = {}

def main():
    seedURLs = [ "https://docs.oracle.com/en/","https://www.oracle.com/corporate/","https://www.csie.ntu.edu.tw/~cjlin/libsvm/index.html"]
    #"https://docs.oracle.com/middleware/jet210/jet/index.html","https://en.wikipedia.org/w/api.php","https://en.wikipedia.org/api/","https://en.wikipedia.org/wiki/Weka_(machine_learning)" ]

    myFrontier = frontier(seedURLs)

    #fill the FQs
    myFrontier.addURLs(seedURLs)

    #right now all BQ are empty so fill them
    #make timestamp heap for the backqueues as well
    for i in range(BACKQUEUES):
      myFrontier.addToBackQueue(i)
      heapq.heappush(myFrontier.heap, [datetime.datetime.now(), i])
    
    #make a list of BACKQUEUES*3 threads
    threads = []
    for i in range(BACKQUEUES*2): 
        th = threading.Thread(target=crawler_thread_task, args=(myFrontier,))
        threads.append(th)
    
    for i in range(BACKQUEUES*2):
      threads[i].start()

    for i in range(BACKQUEUES*2):
      threads[i].join()

if __name__ == "__main__":
    main()

140065837598464
None https://en.wikipedia.org/wiki/Motor_coordination#Muscle_synergies
URL Removed from BQ: https://docs.oracle.com/en/
No URLs in the Frontier
No URLs in the Frontier
1000 URLs Crawled. Stopping Now!
140065837598464
None https://en.wikipedia.org/wiki/Rule-based_machine_learning
140065837598464
None https://en.wikipedia.org/wiki/Learning_classifier_system
140065837598464
None https://en.wikipedia.org/wiki/Artificial_immune_system
140065837598464
None https://en.wikipedia.org/wiki/Rakesh_Agrawal_(computer_scientist)
140065837598464
None https://en.wikipedia.org/wiki/Tomasz_Imieli%C5%84ski
140065837598464
None https://en.wikipedia.org/wiki/Point-of-sale
URL Removed from BQ: None
No URLs in the Frontier
140065837598464
None https://en.wikipedia.org/wiki/Pricing
140065837598464
None https://en.wikipedia.org/wiki/Product_placement
140065837598464
None https://en.wikipedia.org/wiki/Market_basket_analysis
140065837598464
None https://en.wikipedia.org/wiki/Web_usage_mining
1400

KeyboardInterrupt: ignored